Đầu tiên, Người dùng nhắn tin thông qua giao diện Chainlit UI hoặc nền tảng Messenger của Facebook

Tin nhắn sẽ được chuyển tiếp đến BackEnd để xử lý 

BackEnd sẽ tương tác với các database Postgres, Redis
Đồng thời sẽ gọi OpenAI API để hỗ trợ sinh câu trả lời cho người dùng

Data từ các external resource sẽ được xử lý và lưu trong Postgres phục vụ cho quá trình truy xuất

Trong quá trình đưa ra phản hồi , WanDB đóng vai trò như monitor giám sát các hoạt động như gọi OpenAI API , các operator được định nghĩa từ detec_demand, collect_and_retrieval đến generate_response để giám sát quá trình sinh câu trả lời

Cuối cùng , câu trả lời được sinh ra sẽ gửi lại lên Chainlit UI hoặc Messenger Platform cho user

Quá trình ETL dữ liệu :

1. FAQ Data : Các câu hỏi thường gặp từ FPTShop được lưu dưới dạng csv , sau khi tiền xử lý và tạo embeddings được lưu vào bảng faqs trong PostgreSQL
2. Phone, Laptop, Accessory Data : Thông tin sản phẩm điện thoại , máy tính xách tay , phụ kiện được lấy từ API của FPTShop , sau đó được xử lý, phân loại và lưu vào các bảng brands, phones, laptops, accessories

Đi sâu hơn vào luồng hoạt động của chatbot trong BE
Đầu vào của flow chatbot là lịch sử trò chuyện của người dùng, đầu ra là phản hồi cho người dùng

Luồng hoạt động của chatbot được chia thành 3 layer 
Layer đầu tiên , agent phụ trách phân tích thông tin hiện tại của yêu cầu từ người dùng (user request) để xác định loại sản phẩm người dùng cần tư vấn (điện thoại , máy tính xách tay , phụ kiện) hoặc thông tin liên hệ mà người dùng cung cấp nếu có. Từ đó sẽ đưa ra quyết định chuyển tiếp đến layer nào hay cung cấp phản hồi trực tiếp cho người dùng .

Nếu xác định được yêu cầu của người dùng là cần tư vấn về các sản phẩm (điện thoại , máy tính xách tay , phụ  kiện) sẽ chuyển từ layer đầu tiên đến layer thứ hai . Ở layer thứ hai sẽ có một agent được chọn để thực thi thực hiện thu thập yêu cầu và truy xuất thông tin tương ứng với loại sản phẩm mà người dùng quan tâm. Agent sẽ thu thập thông tin từ người dùng và truy xuất thông tin sản phẩm dựa trên yêu cầu thu thập được và chuyển tiếp các thông tin đến layer thứ ba.

Agent ở layer thứ ba sẽ phụ trách nhiệm vụ sinh phản hồi cho người dùng dựa trên thông tin được truyền đến từ các layer trước . Và cuối cùng là truyền đầu ra làm phản hồi cho người dùng



Cụ thể hơn từng layer

Ở layer đầu tiên nhận vào lịch sử trò chuyện với người dùng , sau đó gọi đến OpenAI API và dùng các structured output để phân tích yêu cầu của người dùng và các thông tin liên hệ được người dùng cung cấp nếu có. Sau đó sẽ cập nhật vào bộ nhớ ngắn hạn user_memory , từ đó đưa ra quyết định nên làm gì. Có ba trường hợp sẽ xảy ra

Luồng hoạt động sẽ chuyển tiếp đến layer thứ hai Nếu người dùng quan tâm đến một loại sản phẩm nào đó cụ thể như điện thoại , máy tính cầm tay hoặc phụ kiện mà ở đây người dùng cũng chưa cung cấp thông tin liên hệ. Hoặc cũng có thể chuyển tiếp đến layer thứ ba nếu chưa xác định được nhu cầu của người dùng về sản phẩm hoặc nếu người dùng hỏi các thông tin về chính sách của cửa hàng

Trường hợp người dùng cung cấp thông tin liên hệ chưa được thu thập trước đó thì hệ thống sẽ tạo ra câu trả lời ngay cho người dùng đồng thời gửi email thông báo đến bộ phận chăm sóc khách hàng.

Ở layer thứ hai nhận đầu vào là lịch sử trò chuyện , Agent ở layer này sẽ thực hiện function calling để tiến hành thu thập yêu cầu của người dùng cho việc tìm kiếm hoặc tư vấn một loại sản phẩm cụ thể . Thông tin về yêu cầu của người dùng sẽ được cập nhật vào bộ nhớ ngắn user_memory từ đó thực hiện truy xuất thông tin cần thiết cho câu trả lời từ cơ sở dữ liệu. Từ đó tạo ra các instruction phù hợp cho tạo phản hồi để chuyển đến layer thứ ba. Đồng thời cũng có một số knowledge cần thiết liên quan đến sản phẩm người dùng quan tâm từ việc truy xuất.

Ở layer thứ ba , Agent nhận đầu vào là lịch sử trò chuyện , thông tin các FAQ tương ứng cho mỗi lần người dùng nhắn tin nhắn mới trong đó FAQ được xác định bằng semantic search với tin nhắn mới nhất của người dùng. Đồng thời đầu vào của agent ở layer ba còn gồm các instructions và knowledge được chuyển tiếp từ layer thứ hai. Dựa trên các đầu vào này mà agent ở layer ba sẽ tạo ra phản hồi
